# Shadie demo (v.0.1.1)

In [1]:
import numpy as np
import shadie
import toytree

print("shadie", shadie.__version__)
print("toytree", toytree.__version__)

shadie 0.1.1
toytree 3.0.dev1


### Create a simple chromosome

In [2]:
chrom = shadie.chromosome.explicit({
    (0, 1_200_000): shadie.NONCDS,
    (1_200_001, 1_400_000): shadie.EXON,
    (1_400_001, 1_600_000): shadie.INTRON,
    (1_600_001, 1_800_000): shadie.EXON,
    (1_800_001, 5_000_000): shadie.NONCDS,
})
chrom.draw();

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="700.0px" height="150.0px" viewBox="0 0 700.0 150.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tfe1619825caf439ba4452114e3a09300"> name: noncds
interval: (0, 1200000)
ElementType: g3
coding: False name: exon
interval: (1200001, 1400000)
ElementType: g1
coding: True name: intron
interval: (1400001, 1600000)
ElementType: g2
coding: True name: exon
interval: (1600001, 1800000)
ElementType: g1
coding: True name: noncds
interval: (1800001, 5000000)
ElementType: g3
coding: False 0 1000000 2000000 3000000 4000000 5000000

### Setup models

In [14]:
with shadie.Model() as model_1:
    model_1.initialize(
        chromosome=chrom, 
        sim_time=2000,
        mutation_rate=1e-8, 
        recomb_rate=1e-8,
        file_out="test-1.trees",
    )
    model_1.reproduction.bryophyte_dioicous(500, 500)

In [15]:
with shadie.Model() as model_2:
    model_2.initialize(
        chromosome=chrom, 
        sim_time=2000,
        mutation_rate=1e-8, 
        recomb_rate=1e-8,
        file_out="test-2.trees",
    )
    model_2.reproduction.bryophyte_dioicous(500, 500)

## Set up serial runs
Each simulation is run with a new random seed. When using simulations for data analysis, we suggest saving a list of random seeds and calling values from that list so that simulations can be re-run. 

In [24]:
#save models and seeds to lists
models = [model_1, model_2]
seeds = []

for i in range(0, len(models)):
    seeds.append(np.random.randint(2**31))

[1343067109, 894743138]

In [ ]:
#run simulations serially
for idx, sim in enumerate(models):
    sim.run(seed = seeds[idx])

### Run two populations in parallel

You can also use `ProcessPoolExecutor` to run the simulations in parallel. Each simulation model is run from a different random seed, and writes to a different .trees file path. I also provide the path to my updated `slim` binary as an argument.

In [ ]:
from concurrent.futures import ProcessPoolExecutor

In [31]:
with ProcessPoolExecutor(2) as pool:
    for model in [model_1, model_2]:
        kwargs = {"seed": np.random.randint(2**31)}
        pool.submit(model.run, **kwargs)

### Get post-processor

In [25]:
post = shadie.postsim.TwoSims(
    tree_files=["test-1.trees", "test-2.trees"],
    mut=1e-8,
    recomb=1e-8,
    popsize=500,
    chromosome=chrom, #necessary for some drawing functions, but not for anaylsis tools
)

/Users/elissa/miniconda3/lib/python3.9/site-packages/msprime/ancestry.py:831: TimeUnitsMismatchWarning: The initial_state has time_units=ticks but time is measured in generations in msprime. This may lead to significant discrepancies between the timescales. If you wish to suppress this warning, you can use, e.g., warnings.simplefilter('ignore', msprime.TimeUnitsMismatchWarning)
  warnings.warn(message, TimeUnitsMismatchWarning)


In [26]:
post.tree_sequence.population(2)

Population(id=2, metadata={'description': 'ancestral population simulated by msprime', 'name': 'ancestral', 'slim_id': 2})

### Plot simulation summary

In [27]:
post.draw_tree_sequence(sample=6, seed=333);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1800.0px" height="425.0px" viewBox="0 0 1800.0 425.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t5817da43fc024624b4ef5114a59ffc48"> name: noncds
interval: (0, 1200000)
ElementType: g3
coding: False name: exon
interval: (1200001, 1400000)
ElementType: g1
coding: True name: intron
interval: (1400001, 1600000)
ElementType: g2
coding: True name: exon
interval: (1600001, 1800000)
ElementType: g1
coding: True name: noncds
interval: (1800001, 5000000)
ElementType: g3
coding: False 0 1000000 2000000 3000000 4000000 5000000 <polygon points="50.0,108.49385073285787 62.87878787878788,142.46925366428937 62.87878787878788,312.34626832144664 230.3030303030303,312.34626832144664 230.3030303030303,142.46925366428937 308.31914110256594,108.49385073285787 308.31914110256594,108.49385073285787 230.3030303030303,108.49385073285787 230.3030303030303,108.49385073285787 62.87878787878788,108.49385073285787 62.87878787878788,108.49385073285787 50.0,108.49385073285787" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:0.25;opacity:1.0;stroke:rgb(100%,100%,100%);stroke-opacity:0.5"> idx: 0
interval: (0 - 61399)
tmrca: 4041.17 <polygon points="308.31914110256594,108.49385073285787 230.3030303030303,142.46925366428937 230.3030303030303,312.34626832144664 397.72727272727275,312.34626832144664 397.72727272727275,142.46925366428937 360.1857573587335,108.49385073285787 360.1857573587335,108.49385073285787 397.72727272727275,108.49385073285787 397.72727272727275,108.49385073285787 230.3030303030303,108.49385073285787 230.3030303030303,108.49385073285787 308.31914110256594,108.49385073285787" style="fill:rgb(98.8%,55.3%,38.4%);fill-opacity:0.25;opacity:1.0;stroke:rgb(100%,100%,100%);stroke-opacity:0.5"> idx: 1
interval: (61399 - 73727)
tmrca: 2901.15 <polygon points="360.1857573587335,108.49385073285787 397.72727272727275,142.46925366428937 397.72727272727275,312.34626832144664 565.1515151515152,312.34626832144664 565.1515151515152,142.46925366428937 713.9499062798764,108.49385073285787 713.9499062798764,108.49385073285787 565.1515151515152,108.49385073285787 565.1515151515152,108.49385073285787 397.72727272727275,108.49385073285787 397.72727272727275,108.49385073285787 360.1857573587335,108.49385073285787" style="fill:rgb(55.3%,62.7%,79.6%);fill-opacity:0.25;opacity:1.0;stroke:rgb(100%,100%,100%);stroke-opacity:0.5"> idx: 2
interval: (73727 - 157812)
tmrca: 2901.15 <polygon points="713.9499062798764,108.49385073285787 565.1515151515152,142.46925366428937 565.1515151515152,312.34626832144664 732.5757575757575,312.34626832144664 732.5757575757575,142.46925366428937 1156.250806701246,108.49385073285787 1156.250806701246,108.49385073285787 732.5757575757575,108.49385073285787 732.5757575757575,108.49385073285787 565.1515151515152,108.49385073285787 565.1515151515152,108.49385073285787 713.9499062798764,108.49385073285787" style="fill:rgb(90.6%,54.1%,76.5%);fill-opacity:0.25;opacity:1.0;stroke:rgb(100%,100%,100%);stroke-opacity:0.5"> idx: 3
interval: (157812 - 262941)
tmrca: 3518.68 idx: 4
interval: (262941 - 282189)
tmrca: 3518.68 idx: 5
interval: (282189 - 324682)
tmrca: 3518.68 idx: 6
interval: (324682 - 363190)
tmrca: 5564.8 idx: 7
interval: (363190 - 366501)
tmrca: 4522.49 idx: 8
interval: (366501 - 395769)
tmrca: 4522.49 <polygon points="1715.0875120933795,108.49385073285787 1569.6969696969695,142.46925366428937 1569.6969696969695,312.34626832144664 1737.1212121212122,312.34626832144664 1737.1212121212122,142.46925366428937 1750.0,108.49385073285787 1750.0,108.49385073285787 1737.1212121212122,108.49385073285787 1737.1212121212122,108.49385073285787 156

### Plot individual trees

In [6]:
post.draw_tree(idx=0, sample=[10, 10], seed=123);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="400.0px" height="400.0px" viewBox="0 0 400.0 400.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tafce54edb9e34719a21d270640eafb99"> p0-3 p0-8 p0-0 p0-4 p0-5 p0-6 p0-7 p0-2 p0-1 p0-9 p1-15 p1-19 p1-11 p1-12 p1-16 p1-17 p1-13 p1-14 p1-10 p1-18 id: 0
site: 0
time: 443
mtype: 1 id: 1
site: 1
time: 546
mtype: 0 id: 2
site: 2
time: 1258
mtype: 0 id: 3
site: 3
time: 1054
mtype: 1 id: 4
site: 4
time: 79
mtype: 0 id: 5
site: 5
time: 1679
mtype: 0 id: 6
site: 6
time: 2227
mtype: 0 id: 7
site: 7
time: 758
mtype: 1 id: 8
site: 8
time: 428
mtype: 0 id: 9
site: 9
time: 164
mtype: 1 3000 2000 1000 0 tree in interval 0 (sites 0-78895)

### Calculate statistics

In [7]:
post.stats(sample=10, reps=20)

,mean,CI_5%,CI_95%
theta_0,0.000014,0.000014,0.000015
theta_1,0.000013,0.000012,0.000013
Fst_01,0.706540,0.699409,0.713670
Dist_01,0.000079,0.000078,0.000079
D_Taj_0,0.109832,-0.045565,0.265229
D_Taj_1,0.112428,-0.070684,0.295539


## Access Metadata
You can access the parameters settings from SLiM in the tree sequence metadata

In [11]:
post.tree_sequence.metadata["SLiM"]["user_metadata"]

{'gam_archegonia_per': ['10'],
 'gam_clone_rate': ['0.8'],
 'gam_clones_per': ['10'],
 'gam_female_to_male_ratio': ['0.6666666666666666'],
 'gam_k': ['5000'],
 'gam_maternal_effect': ['0.5'],
 'gam_mutation_rate': ['5e-09'],
 'gam_pop_size': ['500'],
 'gam_random_death_chance': ['0.08'],
 'spo_mutation_rate': ['5e-09'],
 'spo_pop_size': ['500'],
 'spo_random_death_chance': ['0.08'],
 'spo_self_rate_per_egg': ['0.1'],
 'spo_spores_per': ['10']}